## Problem statment
predict whether or not give loan to customer

# Banking

# Asset = Loan product
Housing loan
Personal loan
Vehicle loan
Group loan
Education loan
Credit Card

# Liability
Current account
Savings account
(Casa)

Fixed deposit
Recurring Deposit
(Term Deposits)

With respect to Bank

# NPA
NPA = Non Performing Asset (ex vijay maliya)
NPA = Loan that is defaulted


1. Disbursed Amount = Loan amount given to a customer

2. OSP = Out Standing Principle ()
1 Lakh loan
8000 EMI
OSP
40000 pay
60000 bakaya = Balance = OSP
OSP should be zero at the end of loan cycle


3. DPD = Days past due
DPD = 2
DPD ideally zero

"Defaulted" DPD  > 0


4. PAR
Portfolio at risk
OSP when DPD > 0



5. NPA
Loan account when DPD > 90 days
NPA account

NPA types-

GNPA = Gross NPA (3-5 %) = OSP dafault
NNPA = Net NPA = (0.01 to 0.06 %) = Provisioning Amount subtracted

Bank quality assess, GNPA value

# Credit Risk Types in Banking

DPD (Zero) : NDA (Non delinquint account) = No default account = Timely payment EMI
DPD (0 to 30) : SMA1 (Standard Monitoring Account)
DPD (31 to 60) : SMA2 (Standard Monitoring Account)
DPD (61 to 90) : SMA3 (Standard Monitoring Account)
DPD (90 to 180) : NPA

DPD (>180) : Writen-off (Loan which is not present, remove from loan DB)

written-off to NPA improve = Loan Portfolio quality of the bank will be better = Market sentiment will be good = Stock price will improve


TL = Trade line or lones account


In [146]:
# !pip install statsmodels sklearn scipy os warnings

In [5]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import warnings
import os

In [6]:
from google.colab import drive
drive.mount('/content/drive/')
# /content/drive/MyDrive/dataset

Mounted at /content/drive/


In [7]:
# Load the dataset
a1 = pd.read_excel("/content/drive/MyDrive/case_study1.xlsx")
a2 = pd.read_excel("/content/drive/MyDrive/case_study2.xlsx")

In [8]:

df1 = a1.copy()
df2 = a2.copy()

In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PROSPECTID            51336 non-null  int64  
 1   Total_TL              51336 non-null  int64  
 2   Tot_Closed_TL         51336 non-null  int64  
 3   Tot_Active_TL         51336 non-null  int64  
 4   Total_TL_opened_L6M   51336 non-null  int64  
 5   Tot_TL_closed_L6M     51336 non-null  int64  
 6   pct_tl_open_L6M       51336 non-null  float64
 7   pct_tl_closed_L6M     51336 non-null  float64
 8   pct_active_tl         51336 non-null  float64
 9   pct_closed_tl         51336 non-null  float64
 10  Total_TL_opened_L12M  51336 non-null  int64  
 11  Tot_TL_closed_L12M    51336 non-null  int64  
 12  pct_tl_open_L12M      51336 non-null  float64
 13  pct_tl_closed_L12M    51336 non-null  float64
 14  Tot_Missed_Pmnt       51336 non-null  int64  
 15  Auto_TL            

In [10]:
df1[df1 == -99999].count()

PROSPECTID               0
Total_TL                 0
Tot_Closed_TL            0
Tot_Active_TL            0
Total_TL_opened_L6M      0
Tot_TL_closed_L6M        0
pct_tl_open_L6M          0
pct_tl_closed_L6M        0
pct_active_tl            0
pct_closed_tl            0
Total_TL_opened_L12M     0
Tot_TL_closed_L12M       0
pct_tl_open_L12M         0
pct_tl_closed_L12M       0
Tot_Missed_Pmnt          0
Auto_TL                  0
CC_TL                    0
Consumer_TL              0
Gold_TL                  0
Home_TL                  0
PL_TL                    0
Secured_TL               0
Unsecured_TL             0
Other_TL                 0
Age_Oldest_TL           40
Age_Newest_TL           40
dtype: int64

In [11]:
# Remove nulls
df1 = df1.loc[df1['Age_Oldest_TL'] != -99999]
df1 = df1.loc[df1['Age_Newest_TL'] != -99999]
# df1.shape

In [12]:
df2[df2 == -99999].count()

PROSPECTID                          0
time_since_recent_payment        4291
time_since_first_deliquency     35949
time_since_recent_deliquency    35949
num_times_delinquent                0
                                ...  
GL_Flag                             0
last_prod_enq2                      0
first_prod_enq2                     0
Credit_Score                        0
Approved_Flag                       0
Length: 62, dtype: int64

In [13]:
columns_to_be_removed = []
threshold = 10000
for i in df2.columns:
    if df2.loc[df2[i] == -99999].shape[0] > threshold:
        columns_to_be_removed .append(i)

In [14]:
df2 = df2.drop(columns_to_be_removed, axis =1)

In [15]:
df2.shape

(51336, 54)

In [16]:
for i in df2.columns:
    df2 = df2.loc[ df2[i] != -99999 ]

In [17]:
# Checking common column names
for i in list(df1.columns):
    if i in list(df2.columns):
        print (i)

PROSPECTID


In [18]:
# Merge the two dataframes, inner join so that no nulls are present
df = pd.merge ( df1, df2, how ='inner', left_on = ['PROSPECTID'], right_on = ['PROSPECTID'] )

In [19]:
# check how many columns are categorical
for i in df.columns:
    if df[i].dtype == 'object':
        print(i)

MARITALSTATUS
EDUCATION
GENDER
last_prod_enq2
first_prod_enq2
Approved_Flag


In [20]:
for i in ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']:
    print(pd.crosstab(df[i], df['Approved_Flag']),"\n")

Approved_Flag    P1     P2    P3    P4
MARITALSTATUS                         
Married        4435  18718  4387  3346
Single          473   6734  2053  1918 

Approved_Flag     P1    P2    P3    P4
EDUCATION                             
12TH            1191  7326  1881  1305
GRADUATE        1766  8266  2136  1972
OTHERS           310  1401   327   253
POST-GRADUATE    304  1114   251   229
PROFESSIONAL      44   130    31    14
SSC              761  4518  1104   858
UNDER GRADUATE   532  2697   710   633 

Approved_Flag    P1     P2    P3    P4
GENDER                                
F               451   2946   744   578
M              4457  22506  5696  4686 

Approved_Flag     P1    P2    P3    P4
last_prod_enq2                        
AL               331   842   121    59
CC               308  1315   287   285
ConsumerLoan    1213  9227  3092  2948
HL               237   511    63    19
PL               885  4146  1378  1144
others          1934  9411  1499   809 

Approved_Flag    

In [21]:
# Chi-square test
# measure 1 on chi2 value if chi2 val is greater than tab value than have significant diff
# measure 2 on pval if pval is lessser than alpha than have significant diff
for i in ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']:
    chi2, pval, _, _ = chi2_contingency(pd.crosstab(df[i], df['Approved_Flag']))
    print(i, '---', f"{pval}" , "---",chi2)

MARITALSTATUS --- 3.578180861038862e-233 --- 1076.9871387543772
EDUCATION --- 2.6942265249737532e-30 --- 187.81675366240617
GENDER --- 1.907936100186563e-05 --- 24.56031272141628
last_prod_enq2 --- 0.0 --- 2444.9571510235596
first_prod_enq2 --- 7.84997610555419e-287 --- 1387.5609151031795


In [22]:
# Since all the categorical features have pval <=0.05, we will accept all,Ho rejected

In [23]:
# VIF for numerical columns
numeric_columns = []
for i in df.columns:
    if df[i].dtype != 'object' and i not in ['PROSPECTID','Approved_Flag']:
        numeric_columns.append(i)


In [24]:
# VIF sequentially check

vif_data = df[numeric_columns]
total_columns = vif_data.shape[1]
columns_to_be_kept = []
column_index = 0

In [25]:
total_columns

72

In [26]:
removedCol = []
for i in range (0,total_columns):
    vif_value = variance_inflation_factor(vif_data, column_index)
    print (column_index,'---',vif_value,"    ",i)

    if vif_value <= 6:
        columns_to_be_kept.append( numeric_columns[i] )

        column_index = column_index+1

    else:
        removedCol.append((numeric_columns[i],vif_value))
        vif_data = vif_data.drop([numeric_columns[i] ] , axis=1)



/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf      0


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf      1
0 --- 11.320180023967996      2
0 --- 8.363698035000336      3
0 --- 6.520647877790928      4
0 --- 5.149501618212625      5
1 --- 2.611111040579735      6


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


2 --- inf      7
2 --- 1788.7926256209232      8
2 --- 8.601028256477228      9
2 --- 3.832800792153077      10
3 --- 6.099653381646723      11
3 --- 5.581352009642766      12
4 --- 1.985584353098778      13


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


5 --- inf      14
5 --- 4.80953830281934      15
6 --- 23.270628983464636      16
6 --- 30.595522588100053      17
6 --- 4.384346405965583      18
7 --- 3.0646584155234238      19
8 --- 2.898639771299251      20
9 --- 4.377876915347324      21
10 --- 2.207853583695844      22
11 --- 4.916914200506864      23
12 --- 5.214702030064725      24
13 --- 3.3861625024231476      25
14 --- 7.840583309478997      26
14 --- 5.255034641721434      27


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


15 --- inf      28
15 --- 7.380634506427238      29
15 --- 1.4210050015175733      30
16 --- 8.083255010190316      31
16 --- 1.6241227524040114      32
17 --- 7.257811920140003      33
17 --- 15.59624383268298      34
17 --- 1.825857047132431      35
18 --- 1.5080839450032664      36
19 --- 2.172088834824578      37
20 --- 2.6233975535272274      38
21 --- 2.2959970812106176      39
22 --- 7.360578319196446      40
22 --- 2.1602387773102567      41
23 --- 2.8686288267891467      42
24 --- 6.458218003637272      43
24 --- 2.8474118865638247      44
25 --- 4.753198156284083      45
26 --- 16.22735475594825      46
26 --- 6.424377256363877      47
26 --- 8.887080381808678      48
26 --- 2.3804746142952653      49
27 --- 8.60951347651454      50
27 --- 13.06755093547673      51
27 --- 3.500040056654653      52
28 --- 1.9087955874813773      53
29 --- 17.006562234161628      54
29 --- 10.730485153719197      55
29 --- 2.3538497522950275      56
30 --- 22.10485591513649      57
30 --- 2.797

In [27]:
removedCol  #removed col name and there VIF

[('Total_TL', inf),
 ('Tot_Closed_TL', inf),
 ('Tot_Active_TL', 11.320180023967996),
 ('Total_TL_opened_L6M', 8.363698035000336),
 ('Tot_TL_closed_L6M', 6.520647877790928),
 ('pct_active_tl', inf),
 ('pct_closed_tl', 1788.7926256209232),
 ('Total_TL_opened_L12M', 8.601028256477228),
 ('pct_tl_open_L12M', 6.099653381646723),
 ('Auto_TL', inf),
 ('Consumer_TL', 23.270628983464636),
 ('Gold_TL', 30.595522588100053),
 ('num_times_delinquent', 7.840583309478997),
 ('num_deliq_6mts', inf),
 ('num_deliq_12mts', 7.380634506427238),
 ('num_times_30p_dpd', 8.083255010190316),
 ('num_std', 7.257811920140003),
 ('num_std_6mts', 15.59624383268298),
 ('num_dbt_6mts', 7.360578319196446),
 ('num_lss_6mts', 6.458218003637272),
 ('tot_enq', 16.22735475594825),
 ('CC_enq', 6.424377256363877),
 ('CC_enq_L6m', 8.887080381808678),
 ('PL_enq', 8.60951347651454),
 ('PL_enq_L6m', 13.06755093547673),
 ('enq_L12m', 17.006562234161628),
 ('enq_L6m', 10.730485153719197),
 ('AGE', 22.10485591513649),
 ('pct_of_acti

In [28]:

# check Anova for columns_to_be_kept

from scipy.stats import f_oneway

columns_to_be_kept_numerical = []

for i in columns_to_be_kept:
    a = list(df[i])
    b = list(df['Approved_Flag'])

    group_P1 = [value for value, group in zip(a, b) if group == 'P1']
    group_P2 = [value for value, group in zip(a, b) if group == 'P2']
    group_P3 = [value for value, group in zip(a, b) if group == 'P3']
    group_P4 = [value for value, group in zip(a, b) if group == 'P4']


    f_statistic, p_value = f_oneway(group_P1, group_P2, group_P3, group_P4)

    if p_value <= 0.05:
        columns_to_be_kept_numerical.append(i)

# feature selection is done for cat and num features


In [29]:
# listing all the final features
features = columns_to_be_kept_numerical + ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']
df = df[features + ['Approved_Flag']]

In [30]:
list(df.columns)

['pct_tl_open_L6M',
 'pct_tl_closed_L6M',
 'Tot_TL_closed_L12M',
 'pct_tl_closed_L12M',
 'Tot_Missed_Pmnt',
 'CC_TL',
 'Home_TL',
 'PL_TL',
 'Secured_TL',
 'Unsecured_TL',
 'Other_TL',
 'Age_Oldest_TL',
 'Age_Newest_TL',
 'time_since_recent_payment',
 'max_recent_level_of_deliq',
 'num_deliq_6_12mts',
 'num_times_60p_dpd',
 'num_std_12mts',
 'num_sub',
 'num_sub_6mts',
 'num_sub_12mts',
 'num_dbt',
 'num_dbt_12mts',
 'num_lss',
 'recent_level_of_deliq',
 'CC_enq_L12m',
 'PL_enq_L12m',
 'time_since_recent_enq',
 'enq_L3m',
 'NETMONTHLYINCOME',
 'Time_With_Curr_Empr',
 'CC_Flag',
 'PL_Flag',
 'pct_PL_enq_L6m_of_ever',
 'pct_CC_enq_L6m_of_ever',
 'HL_Flag',
 'GL_Flag',
 'MARITALSTATUS',
 'EDUCATION',
 'GENDER',
 'last_prod_enq2',
 'first_prod_enq2',
 'Approved_Flag']

## **# Label encoding for the categorical features**

In [31]:
# ['MARITALSTATUS', 'EDUCATION', 'GENDER' , 'last_prod_enq2' ,'first_prod_enq2']

df[['MARITALSTATUS', 'EDUCATION', 'GENDER' , 'last_prod_enq2' ,'first_prod_enq2']].nunique()

MARITALSTATUS      2
EDUCATION          7
GENDER             2
last_prod_enq2     6
first_prod_enq2    6
dtype: int64

In [32]:
# Ordinal feature -- EDUCATION
# SSC            : 1
# 12TH           : 2
# GRADUATE       : 3
# UNDER GRADUATE : 3
# POST-GRADUATE  : 4
# OTHERS         : 1
# PROFESSIONAL   : 3

In [33]:
df.loc[df['EDUCATION'] == 'SSC',['EDUCATION']]              = 1
df.loc[df['EDUCATION'] == '12TH',['EDUCATION']]             = 2
df.loc[df['EDUCATION'] == 'GRADUATE',['EDUCATION']]         = 3
df.loc[df['EDUCATION'] == 'UNDER GRADUATE',['EDUCATION']]   = 3
df.loc[df['EDUCATION'] == 'POST-GRADUATE',['EDUCATION']]    = 4
df.loc[df['EDUCATION'] == 'OTHERS',['EDUCATION']]           = 1
df.loc[df['EDUCATION'] == 'PROFESSIONAL',['EDUCATION']]     = 3


In [34]:
# prompt: from the above cell create a dic which have key as value after ==  and value is assignment value

education_mapping = {
    'SSC': 1,
    '12TH': 2,
    'GRADUATE': 3,
    'UNDER GRADUATE': 3,
    'POST-GRADUATE': 4,
    'OTHERS': 1,
    'PROFESSIONAL': 3
}


In [35]:
# prompt: export education_mapping as json file

import json

with open('/content/education_mapping.json', 'w') as f:
  json.dump(education_mapping, f)


In [36]:
df['EDUCATION'].value_counts()
df['EDUCATION'] = df['EDUCATION'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   pct_tl_open_L6M            42064 non-null  float64
 1   pct_tl_closed_L6M          42064 non-null  float64
 2   Tot_TL_closed_L12M         42064 non-null  int64  
 3   pct_tl_closed_L12M         42064 non-null  float64
 4   Tot_Missed_Pmnt            42064 non-null  int64  
 5   CC_TL                      42064 non-null  int64  
 6   Home_TL                    42064 non-null  int64  
 7   PL_TL                      42064 non-null  int64  
 8   Secured_TL                 42064 non-null  int64  
 9   Unsecured_TL               42064 non-null  int64  
 10  Other_TL                   42064 non-null  int64  
 11  Age_Oldest_TL              42064 non-null  int64  
 12  Age_Newest_TL              42064 non-null  int64  
 13  time_since_recent_payment  42064 non-null  int

<ipython-input-36-c1e7fd256653>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EDUCATION'] = df['EDUCATION'].astype(int)


In [37]:
labels = dict()
for i in ['MARITALSTATUS','GENDER', 'last_prod_enq2' ,'first_prod_enq2']:
  labels[i] = list(df[i].unique()
)

In [38]:
# list(df['MARITALSTATUS'].unique())
labels

{'MARITALSTATUS': ['Married', 'Single'],
 'GENDER': ['M', 'F'],
 'last_prod_enq2': ['PL', 'ConsumerLoan', 'AL', 'CC', 'others', 'HL'],
 'first_prod_enq2': ['PL', 'ConsumerLoan', 'others', 'AL', 'HL', 'CC']}

In [39]:
# prompt: write a code to save labels dictionary in json file

import json

with open('/content/drive/My Drive/CatColumnsAndlabels.json', 'w') as f:
  json.dump(labels, f)


In [40]:
import json

# Open the JSON file
with open('/content/drive/My Drive/CatColumnsAndlabels.json', 'r') as f:
  data = json.load(f)

# Store the values in a dictionary
labels = data
labels


{'MARITALSTATUS': ['Married', 'Single'],
 'GENDER': ['M', 'F'],
 'last_prod_enq2': ['PL', 'ConsumerLoan', 'AL', 'CC', 'others', 'HL'],
 'first_prod_enq2': ['PL', 'ConsumerLoan', 'others', 'AL', 'HL', 'CC']}

In [41]:
# getting OHE data for Nominal data
df_encoded = pd.get_dummies(df, columns=['MARITALSTATUS','GENDER', 'last_prod_enq2' ,'first_prod_enq2'])
df_encoded.head()

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,...,last_prod_enq2_ConsumerLoan,last_prod_enq2_HL,last_prod_enq2_PL,last_prod_enq2_others,first_prod_enq2_AL,first_prod_enq2_CC,first_prod_enq2_ConsumerLoan,first_prod_enq2_HL,first_prod_enq2_PL,first_prod_enq2_others
0,0.000,0.0,0,0.000,0,0,0,4,1,4,...,False,False,True,False,False,False,False,False,True,False
1,0.000,0.0,0,0.000,0,0,0,0,0,1,...,True,False,False,False,False,False,True,False,False,False
2,0.125,0.0,0,0.000,1,0,0,0,2,6,...,True,False,False,False,False,False,False,False,False,True
3,0.000,0.0,0,0.000,0,0,0,0,3,0,...,False,False,False,False,True,False,False,False,False,False
4,0.000,0.0,1,0.167,0,0,0,0,6,0,...,True,False,False,False,False,False,False,False,True,False


In [42]:
df_encoded.columns = df_encoded.columns.str.lstrip('_')
df_encoded.columns

Index(['pct_tl_open_L6M', 'pct_tl_closed_L6M', 'Tot_TL_closed_L12M',
       'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'CC_TL', 'Home_TL', 'PL_TL',
       'Secured_TL', 'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL',
       'Age_Newest_TL', 'time_since_recent_payment',
       'max_recent_level_of_deliq', 'num_deliq_6_12mts', 'num_times_60p_dpd',
       'num_std_12mts', 'num_sub', 'num_sub_6mts', 'num_sub_12mts', 'num_dbt',
       'num_dbt_12mts', 'num_lss', 'recent_level_of_deliq', 'CC_enq_L12m',
       'PL_enq_L12m', 'time_since_recent_enq', 'enq_L3m', 'NETMONTHLYINCOME',
       'Time_With_Curr_Empr', 'CC_Flag', 'PL_Flag', 'pct_PL_enq_L6m_of_ever',
       'pct_CC_enq_L6m_of_ever', 'HL_Flag', 'GL_Flag', 'EDUCATION',
       'Approved_Flag', 'MARITALSTATUS_Married', 'MARITALSTATUS_Single',
       'GENDER_F', 'GENDER_M', 'last_prod_enq2_AL', 'last_prod_enq2_CC',
       'last_prod_enq2_ConsumerLoan', 'last_prod_enq2_HL', 'last_prod_enq2_PL',
       'last_prod_enq2_others', 'first_prod_enq2_AL',

# **Machine Learing model fitting**

In [43]:
y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [44]:
import pickle
finalColumnsName = x.columns
with open('/content/drive/MyDrive/finalColumnsName.pkl', 'wb') as f:
    pickle.dump(finalColumnsName, f)

In [45]:
with open('/content/drive/MyDrive/finalColumnsName.pkl', 'rb') as f:
    finalColumnsName = pickle.load(f)
finalColumnsName

Index(['pct_tl_open_L6M', 'pct_tl_closed_L6M', 'Tot_TL_closed_L12M',
       'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'CC_TL', 'Home_TL', 'PL_TL',
       'Secured_TL', 'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL',
       'Age_Newest_TL', 'time_since_recent_payment',
       'max_recent_level_of_deliq', 'num_deliq_6_12mts', 'num_times_60p_dpd',
       'num_std_12mts', 'num_sub', 'num_sub_6mts', 'num_sub_12mts', 'num_dbt',
       'num_dbt_12mts', 'num_lss', 'recent_level_of_deliq', 'CC_enq_L12m',
       'PL_enq_L12m', 'time_since_recent_enq', 'enq_L3m', 'NETMONTHLYINCOME',
       'Time_With_Curr_Empr', 'CC_Flag', 'PL_Flag', 'pct_PL_enq_L6m_of_ever',
       'pct_CC_enq_L6m_of_ever', 'HL_Flag', 'GL_Flag', 'EDUCATION',
       'MARITALSTATUS_Married', 'MARITALSTATUS_Single', 'GENDER_F', 'GENDER_M',
       'last_prod_enq2_AL', 'last_prod_enq2_CC', 'last_prod_enq2_ConsumerLoan',
       'last_prod_enq2_HL', 'last_prod_enq2_PL', 'last_prod_enq2_others',
       'first_prod_enq2_AL', 'first_prod_enq2

In [46]:
finalColumnsName.shape

(54,)

In [47]:
# 1. Random Forest


rf_classifier = RandomForestClassifier(n_estimators = 200, random_state=42)
rf_classifier.fit(x_train, y_train)

y_pred = rf_classifier.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

print ()
print(f'Accuracy: {accuracy}')
print ()

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()







Accuracy: 0.7636990372043266

Class p1:
Precision: 0.8370457209847597
Recall: 0.7041420118343196
F1 Score: 0.7648634172469203

Class p2:
Precision: 0.7957519116397621
Recall: 0.9282457879088206
F1 Score: 0.8569075937785909

Class p3:
Precision: 0.4423380726698262
Recall: 0.21132075471698114
F1 Score: 0.28600612870275793

Class p4:
Precision: 0.7178502879078695
Recall: 0.7269193391642371
F1 Score: 0.7223563495895703



In [48]:


# 2. xgboost

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

xgb_classifier = xgb.XGBClassifier(objective='multi:softmax',  num_class=4)



y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)



xgb_classifier.fit(x_train, y_train)
y_pred = xgb_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy:.2f}')
print ()

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()








Accuracy: 0.78

Class p1:
Precision: 0.823906083244397
Recall: 0.7613412228796844
F1 Score: 0.7913890312660173

Class p2:
Precision: 0.8255418233924413
Recall: 0.913577799801784
F1 Score: 0.8673315769665036

Class p3:
Precision: 0.4756380510440835
Recall: 0.30943396226415093
F1 Score: 0.3749428440786465

Class p4:
Precision: 0.7342386032977691
Recall: 0.7356656948493683
F1 Score: 0.7349514563106796



In [49]:
# 3. Decision Tree
from sklearn.tree import DecisionTreeClassifier


y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


dt_model = DecisionTreeClassifier(max_depth=20, min_samples_split=10)
dt_model.fit(x_train, y_train)
y_pred = dt_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print ()
print(f"Accuracy: {accuracy:.2f}")
print ()

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()







Accuracy: 0.71

Class p1:
Precision: 0.7208619000979432
Recall: 0.7258382642998028
F1 Score: 0.7233415233415234

Class p2:
Precision: 0.8086719813338519
Recall: 0.8243805748265609
F1 Score: 0.8164507263447193

Class p3:
Precision: 0.34365079365079365
Recall: 0.3267924528301887
F1 Score: 0.3350096711798839

Class p4:
Precision: 0.6481294236602629
Recall: 0.6229348882410107
F1 Score: 0.6352824578790882




### xgboost is giving me best results
### We will further finetune it

In [58]:

# Hyperparameter tuning in xgboost
from sklearn.model_selection import GridSearchCV
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

# Define the XGBClassifier with the initial set of hyperparameters
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=4)


# Define the parameter grid for hyperparameter tuning

param_grid = {
    'n_estimators': [100,200],
    'max_depth': [4,6,8],
    'learning_rate': [0.01,0.1,0.2],
    'alpha': [0.1,0.5,1,10]
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model with the best hyperparameters on the test set
best_model = grid_search.best_estimator_
accuracy = best_model.score(x_test, y_test)
print("Test Accuracy:", accuracy)

# Best Hyperparameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200}

Best Hyperparameters: {'alpha': 1, 'learning_rate': 0.2, 'max_depth': 4, 'n_estimators': 100}
Test Accuracy: 0.7786758587899679


In [1]:
# prompt: from above cell gridsearch cv print all parameter and result also add accuracy and loss

# Print the results of the grid search
print("GridSearchCV Results:")
print()
print("Parameters:", grid_search.cv_results_['params'])
print("Mean Test Accuracy:", grid_search.cv_results_['mean_test_score'])
print("Standard Deviation:", grid_search.cv_results_['std_test_score'])
print()

# Print the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Test Accuracy:", grid_search.best_score_)

# Evaluate the best model on the test set
y_pred = best_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
loss = log_loss(y_test, best_model.predict_proba(x_test))
print("Test Accuracy:", accuracy)
print("Test Loss:", loss)


GridSearchCV Results:



NameError: name 'grid_search' is not defined

In [52]:
# Getting best results with these-
# colsample_bytree  0.7
# learning_rate     0.2
# max_depth         3
# alpha             10.0
# n_estimators      200

In [53]:
# Retrain on the new parameters
model = xgb.XGBClassifier(objective='multi:softmax',
                         colsample_bytree = 0.7,
                         learning_rate    = 0.3,
                         max_depth        = 5,
                         alpha            = 0.3,
                         n_estimators     = 200,
                         eval_metric = 'merror',
                         num_class = 4)


model.fit(x_train, y_train)

XGBClassifier(alpha=0.3, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='merror',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.3, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_class=4, ...)

In [54]:
from sklearn.metrics import mean_squared_error

# Predicting on the testing set
y_pred = model.predict(x_test)

# Calculate loss metrics
print()
r2 = r2_score(y_test, y_pred)
print('R-squared:', r2)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('RMSE:', rmse)
stddev = y_test.std()
print('Stddev difference', rmse/stddev)


R-squared: 0.5879347277270921
RMSE: 0.5324376368710059
Stddev difference 0.6419231046417537


In [55]:
model.score(x_test,y_test)

0.7713063116605253

In [56]:
# save the model
import pickle
filename = 'eps_v1.pkl'
pickle.dump(model, open(filename,'wb'))


# load_model = pickle.load(open(filename,'rb'))


# arg = x_train[:2]
# load_model.predict(arg)


# ROCE (%)
# 1.91


# CASA (%)
# 39.47


# Return on Equity / Networth (%)
# 14.36


# Non-Interest Income/Total Assets (%)
# 0.68


# Operating Profit/Total Assets (%)
# 0.27


# Operating Expenses/Total Assets (%)
# 1.68

# Interest Expenses/Total Assets (%)
# 3.3


# Face_value
# 2


# Basic EPS (Rs.)
# 27.28

In [57]:
model.predict(x_test)

array([1, 3, 1, ..., 2, 1, 3], dtype=int32)